In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline
%run helpers.py

In [21]:
# Q1- Make Trackbar for the following images to tune the parameters of canny detection.
cv2.namedWindow('Image')

def Nothing(x):
    pass

cv2.createTrackbar("Threshold 1", 'Image', 0, 200, Nothing)
cv2.createTrackbar("Threshold 2", 'Image', 0, 200, Nothing)


while True:

    CANNY_THRESHOLD_1 = cv2.getTrackbarPos("Threshold 1", 'Image') # try: 50 - 100      # Typical: 50
    CANNY_THRESHOLD_2 = cv2.getTrackbarPos("Threshold 2", 'Image') # try: 100 - 200     # Typical: 150
    def canny_edges(gray): # https://docs.opencv.org/3.4/d7/de1/tutorial_js_canny.html
        kernel_size = 5 #kernel size for Gaussian smoothing / blurring
        blur_gray= cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)#Gaussian smoothing / blurring

        # finding edges - Canny Edge detection (strong gradient between adjacent pixels)
        edges = cv2.Canny(blur_gray, CANNY_THRESHOLD_1, CANNY_THRESHOLD_2)
        return(edges)


    #HOUGH LINES PARAMETERS
    RHO = 1                 # try: 1 - 4 (0.5 increments)  
    THETA = np.pi/180       # Usually this is Ok
    MIN_VOTES = 10          # try: 10 - 50                  # Typical: 30
    MIN_LINE_LEN = 5 
    MAX_LINE_GAP = 25 

    def hough_lines(img):
        lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]),
                                minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
        return lines

    # Read the image:
    image = cv2.imread('test_images/solidWhiteCurve.jpg') #read in
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)#grayscale conversion

    # 5.   Get the edges (Canny)
    edges_img = canny_edges(gray_img)

    cv2.imshow('Image', edges_img)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

print('Destroying....')
cv2.destroyAllWindows()


Destroying....


## Observation:
#### - the more values of thresholds (1,2) the little edges you see.

In [26]:
# Q2- The out of question 1 make it as input for question 2 after make region of
# interest then make trackbar for houghlines parameters (threshold ,min_line_length
# max_line_gap) and then draw 2 line on image.
cv2.namedWindow('Image')

def Nothing(x):
    pass

cv2.createTrackbar("Threshold 1", 'Image', 0, 200, Nothing)
cv2.createTrackbar("Threshold 2", 'Image', 0, 200, Nothing)
cv2.createTrackbar("Threshold", 'Image', 0, 200, Nothing)
cv2.createTrackbar("Min Line Length", 'Image', 0, 200, Nothing)
cv2.createTrackbar("Max Line Gap", 'Image', 1, 200, Nothing)

while True:

    CANNY_THRESHOLD_1 = cv2.getTrackbarPos("Threshold 1", 'Image') # try: 50 - 100      # Typical: 50
    CANNY_THRESHOLD_2 = cv2.getTrackbarPos("Threshold 2", 'Image') # try: 100 - 200     # Typical: 150
    def canny_edges(gray): # https://docs.opencv.org/3.4/d7/de1/tutorial_js_canny.html
        kernel_size = 5 #kernel size for Gaussian smoothing / blurring
        blur_gray= cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)#Gaussian smoothing / blurring

        # finding edges - Canny Edge detection (strong gradient between adjacent pixels)
        edges = cv2.Canny(blur_gray, CANNY_THRESHOLD_1, CANNY_THRESHOLD_2)
        return(edges)


    #HOUGH LINES PARAMETERS
    RHO = 1                 # try: 1 - 4 (0.5 increments)  
    THETA = np.pi/180       # Usually this is Ok
    MIN_VOTES = cv2.getTrackbarPos("Threshold", 'Image')          # try: 10 - 50                  # Typical: 30
    MIN_LINE_LEN = cv2.getTrackbarPos("Min Line Length", 'Image')
    MAX_LINE_GAP = cv2.getTrackbarPos("Max Line Gap", 'Image')

    def hough_lines(img):
        lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]),
                                minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
        return lines

    # Read the image:
    image = cv2.imread('test_images/solidWhiteCurve.jpg') #read in
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)#grayscale conversion

    # 5.   Get the edges (Canny)
    edges_img = canny_edges(gray_img)

    # 6.   Remove unwanted parts of the image (too many edges) ROI
    ROI_edges_img = helpers_masked_edges(edges_img)

    # 7.   Get the lines (from edges - Hough lines)
    lines = hough_lines(ROI_edges_img)
    hough_lines_image = helpers_draw_lines(lines, ROI_edges_img)

    # 8.   Formulate 2 lane lines
    lanes = helpers_formulate_lanes(lines, ROI_edges_img)

    # 9.   Create an image with those 2 lines
    lanes_image = helpers_draw_lines(lanes, ROI_edges_img)

    # 10.   Combine the orginal frame with the 2 lines
    final_image = cv2.addWeighted(image, 0.8, lanes_image, 1, 0) 

    cv2.imshow('Image Show', final_image[:,:,::-1])
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

print('Destroying....')
cv2.destroyAllWindows()


Destroying....


## Observation:
- the more values of thresholds (1,2) the little edges you see.
- increasing threshold value may cause some lines to disappear.
- changing in (max line gap, min line length) affect the result